In [1]:
import pandas as pd

#Preprocessing
##Checking Null Values


In [2]:
docs =[r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_0_500000.csv',r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_500000_1000000.csv',r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_1000000_1006029.csv' ]
for doc in docs:
    df = pd.read_csv(doc)
    print(f'Checking for file: {doc}')
    print(df.isnull().sum())
    print('\n')

Checking for file: S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_0_500000.csv
date              0
state             0
district          0
pincode           0
age_0_5           0
age_5_17          0
age_18_greater    0
dtype: int64


Checking for file: S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_500000_1000000.csv
date              0
state             0
district          0
pincode           0
age_0_5           0
age_5_17          0
age_18_greater    0
dtype: int64


Checking for file: S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_1000000_1006029.csv
date              0
state             0
district          0
pincode           0
age_0_5           0
age_5_17          0
age_18_greater    0
dtype: int64




#Merge csv's

In [3]:
#Merge all three csv files into a single csv file
import pandas as pd
df_list = []
for doc in docs:
    df = pd.read_csv(doc)
    df_list.append(df)
merged_df = pd.concat(df_list, ignore_index=True)
merged_df.to_csv(r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_merged.csv', index=False)


##Infogrpahics

In [4]:
#Check for number of rows and columns in the merged csv file
df = pd.read_csv(r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_merged.csv')
print(f'Number of rows: {df.shape[0]}')
print(f'Number of columns: {df.shape[1]}')


Number of rows: 1006029
Number of columns: 7


In [5]:
# Parse the date column
import pandas as pd
df = pd.read_csv(r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_merged.csv')
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y').dt.strftime('%d-%m-%Y')
df.to_csv(r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_merged.csv', index=False)
print(df["date"].head())
print(df["date"].dtype)

0    02-03-2025
1    09-03-2025
2    09-03-2025
3    09-03-2025
4    09-03-2025
Name: date, dtype: object
object


In [6]:
# Ensure consistent categorical values
import pandas as pd
df = pd.read_csv(r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_merged.csv')

# Standardize state and district to title case for consistency
df['state'] = df['state'].str.title()
df['district'] = df['district'].str.title()

# Ensure pincode is string and padded to 6 digits
df['pincode'] = df['pincode'].astype(str).str.zfill(6)

# Drop invalid state entries
df.drop(df[df['state'] == '100000'].index, inplace=True)

# Check unique values to verify consistency
print("Unique states:")
print(df['state'].unique()[:10])  # Show first 10
print("\nUnique districts (first 10):")
print(df['district'].unique()[:10])

# Save back to CSV
df.to_csv(r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_merged.csv', index=False)
print("\nCategorical values standardized and saved.")

Unique states:
['Meghalaya' 'Karnataka' 'Uttar Pradesh' 'Bihar' 'Maharashtra' 'Haryana'
 'Rajasthan' 'Punjab' 'Delhi' 'Madhya Pradesh']

Unique districts (first 10):
['East Khasi Hills' 'Bengaluru Urban' 'Kanpur Nagar' 'Aligarh' 'Sitamarhi'
 'Bahraich' 'Firozabad' 'Purbi Champaran' 'Maharajganj' 'Aurangabad']

Categorical values standardized and saved.


##Correcting state and districts names

In [7]:
# List All States and Their Districts
import pandas as pd

# Load the merged dataset
df = pd.read_csv(r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_merged.csv')

print('Total rows:', len(df))

# Group by state and get unique districts
state_districts = df.groupby('state')['district'].unique().reset_index()

# Create a comprehensive list of all state-district pairs
all_pairs = []
for _, row in state_districts.iterrows():
    state = row['state']
    districts = row['district']
    for district in districts:
        all_pairs.append({'state': state, 'district': district})

# Create DataFrame
state_district_df = pd.DataFrame(all_pairs)

# Sort by state and district
state_district_df = state_district_df.sort_values(['state', 'district']).reset_index(drop=True)

# Save to CSV
output_path = r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\state_district_mapping.csv'
state_district_df.to_csv(output_path, index=False)

print(f"\nState-District Mapping saved to: {output_path}")
print(f"Total unique state-district pairs: {len(state_district_df)}")
print(f"Total unique states: {state_district_df['state'].nunique()}")
print(f"Total unique districts: {state_district_df['district'].nunique()}")

print("\nFirst 20 state-district pairs:")
print(state_district_df.head(20))

print("\nStates with most districts:")
district_counts = state_district_df.groupby('state')['district'].count().sort_values(ascending=False)
print(district_counts.head(10))

print("\nSample districts per state (first 5 states):")
for state in state_district_df['state'].unique()[:5]:
    districts = state_district_df[state_district_df['state'] == state]['district'].tolist()
    print(f"{state}: {districts[:10]}{'...' if len(districts) > 10 else ''}")  # Show first 10 districts

Total rows: 1006007

State-District Mapping saved to: S:\UIDAI_Hackathon\api_data_aadhar_enrolment\state_district_mapping.csv
Total unique state-district pairs: 1045
Total unique states: 48
Total unique districts: 964

First 20 state-district pairs:
                          state                      district
0     Andaman & Nicobar Islands                      Andamans
1     Andaman & Nicobar Islands                      Nicobars
2     Andaman & Nicobar Islands                 South Andaman
3   Andaman And Nicobar Islands                       Nicobar
4   Andaman And Nicobar Islands      North And Middle Andaman
5   Andaman And Nicobar Islands                 South Andaman
6                Andhra Pradesh                      Adilabad
7                Andhra Pradesh         Alluri Sitharama Raju
8                Andhra Pradesh                    Anakapalli
9                Andhra Pradesh                     Anantapur
10               Andhra Pradesh                    Ananthapur
11    

#CrossChecked from govt website

In [8]:
state_map = {
    "Andaman & Nicobar Islands": "Andaman and Nicobar Islands",
    "Andaman And Nicobar Islands": "Andaman and Nicobar Islands",
    "Andhra Pradesh": "Andhra Pradesh",
    "Arunachal Pradesh": "Arunachal Pradesh",
    "Assam": "Assam",
    "Bihar": "Bihar",
    "Dadra & Nagar Haveli": "Dadra and Nagar Haveli and Daman and Diu",
    "Dadra And Nagar Haveli": "Dadra and Nagar Haveli and Daman and Diu",
    "Daman & Diu": "Dadra and Nagar Haveli and Daman and Diu",
    "Daman And Diu": "Dadra and Nagar Haveli and Daman and Diu",
    "Dadra And Nagar Haveli And Daman And Diu": "Dadra and Nagar Haveli and Daman and Diu",
    "The Dadra And Nagar Haveli And Daman And Diu": "Dadra and Nagar Haveli and Daman and Diu",
    "Delhi": "Delhi",
    "Goa": "Goa",
    "Gujarat": "Gujarat",
    "Haryana": "Haryana",
    "Himachal Pradesh": "Himachal Pradesh",
    "Jammu & Kashmir": "Jammu and Kashmir",
    "Jammu And Kashmir": "Jammu and Kashmir",
    "Jharkhand": "Jharkhand",
    "Karnataka": "Karnataka",
    "Kerala": "Kerala",
    "Ladakh": "Ladakh",
    "Lakshadweep": "Lakshadweep",
    "Madhya Pradesh": "Madhya Pradesh",
    "Maharashtra": "Maharashtra",
    "Manipur": "Manipur",
    "Meghalaya": "Meghalaya",
    "Mizoram": "Mizoram",
    "Nagaland": "Nagaland",
    "Orissa": "Odisha",
    "Odisha": "Odisha",
    "Pondicherry": "Puducherry",
    "Puducherry": "Puducherry",
    "Punjab": "Punjab",
    "Rajasthan": "Rajasthan",
    "Sikkim": "Sikkim",
    "Tamil Nadu": "Tamil Nadu",
    "Telangana": "Telangana",
    "Tripura": "Tripura",
    "Uttar Pradesh": "Uttar Pradesh",
    "Uttarakhand": "Uttarakhand",
    "West  Bengal": "West Bengal",
    "West Bangal": "West Bengal",
    "Westbengal": "West Bengal",
    "West Bengal": "West Bengal",
    "Chandigarh": "Chandigarh",
    "Chhattisgarh": "Chhattisgarh",
}

district_map = {
    "Andaman and Nicobar Islands": {
        "Nicobars": "Nicobar",
    },

    "Andhra Pradesh": {
        "Anantapur": "Ananthapuramu",
        "Ananthapur": "Ananthapuramu",
        "Ananthapuramu": "Ananthapuramu",
        "Spsr Nellore": "Sri Potti Sriramulu Nellore",
        "Nellore":"Sri Potti Sriramulu Nellore",
        "Sri Potti Sriramulu Nellore": "Sri Potti Sriramulu Nellore",
        "Visakhapatanam": "Visakhapatnam",
        "Visakhapatnam": "Visakhapatnam",
        "Y. S. R":"Cuddapah"
    },
    "Telangana": {
        "Jagitial": "Jagtial",
        "Jangoan": "Jangaon",
        "Karim Nagar": "Karimnagar",
        "K.V. Rangareddy": "Rangareddy",
        "Mahabub Nagar": "Mahabubnagar",
        "Mahbubnagar": "Mahabubnagar",
        "Medchal Malkajgiri": "Medchal-Malkajgiri",
        "Medchal?Malkajgiri": "Medchal-Malkajgiri",
        "Medchal-Malkajgiri": "Medchal-Malkajgiri",
        "Medchal−Malkajgiri": "Medchal-Malkajgiri",
        "Medchalâˆ’Malkajgiri": "Medchal-Malkajgiri",
        "Ranga Reddy": "Rangareddy",
        "Rangareddy": "Rangareddy",
        "Warangal (Urban)": "Warangal Urban",

    },
    "Assam": {
        "Sibsagar": "Sivasagar",
    },
    "Bihar": {
        "Aurangabad": "Aurangabad(BH)",
        "Aurangabad(Bh)": "Aurangabad(BH)",
        "Monghyr": "Munger",
        "Purba Champaran": "Purbi Champaran",
        "Purnea": "Purnia",
        "Samstipur":"Samastipur",
        "Sheikpura":"Sheikhpura",
        "West Champaran": "Paschim Champaran",
        "Pashchim Champaran":"Paschim Champaran",
        "East Champaran":"Purbi Champaran"
    },
    "Chhattisgarh": {
        "Manendragarhâ€“Chirmiriâ€“Bharatpur": "Manendragarh-Chirmiri-Bharatpur",
        "Mohalla-Manpur-Ambagarh Chowki":"Mohla-Manpur-Ambagarh Chouki",
        "Janjgir - Champa": "Janjgir-Champa",
        "Janjgir Champa": "Janjgir-Champa",
        "Gaurella Pendra Marwahi":"Gaurela-Pendra-Marwahi",
    },
    "Dadra and Nagar Haveli and Daman and Diu": {
        "Dadra And Nagar Haveli": "Dadra And Nagar Haveli",
        "Dadra & Nagar Haveli": "Dadra And Nagar Haveli",
    },
    "Delhi": {
        "North East": "North East Delhi",
        "North East   *": "North East Delhi",

    },
    "Gujarat": {
        "Ahmadabad": "Ahmedabad",
        "Banas Kantha": "Banaskantha",
        "Dohad": "Dahod",
        "Panch Mahals": "Panch Mahals",
        "Panchmahals": "Panch Mahals",
        "Sabar Kantha": "Sabarkantha",
        "Surendra Nagar": "Surendranagar",
    },
    "Haryana": {
        "Gurgaon": "Gurugram",
        "Jhajjar *": "Jhajjar",
        "Mewat": "Nuh",
        "Yamuna Nagar": "Yamunanagar",
    },
    "Himachal Pradesh": {
        "Lahul & Spiti": "Lahaul and Spiti",
        "Lahul And Spiti": "Lahaul and Spiti",
    },
    "Jammu and Kashmir": {
        "Badgam": "Budgam",
        "Bandipur": "Bandipore",
        "Rajauri": "Rajouri",
        "Shupiyan": "Shopian",
    },
    "Jharkhand": {
        "Bokaro *": "Bokaro",
        "East Singhbum": "East Singhbhum",
        "Garhwa *": "Garhwa",
        "Hazaribag": "Hazaribagh",
        "Kodarma": "Koderma",
        "Pakaur": "Pakur",
        "Palamu": "Palamau",
        "Pashchimi Singhbhum": "West Singhbhum",
        "Purbi Singhbhum": "East Singhbhum",
        "Sahibganj": "Sahebganj",
    },
    "Karnataka": {
        "Bagalkot *": "Bagalkot",
        "Bangalore": "Bengaluru Urban",
        "Bangalore Rural": "Bengaluru Rural",
        "Belgaum": "Belagavi",
        "Bellary": "Ballari",
        "Bijapur": "Vijayapura",
        "Bijapur(Kar)": "Vijayapura",
        "Chamrajanagar": "Chamarajanagar",
        "Chamrajnagar": "Chamarajanagar",
        "Chickmagalur": "Chikkamagaluru",
        "Chikmagalur": "Chikkamagaluru",
        "Davangere": "Davanagere",
        "Gadag *": "Gadag",
        "Gulbarga": "Kalaburagi",
        "Hasan": "Hassan",
        "Haveri *": "Haveri",
        "Mysore": "Mysuru",
        "Ramanagar": "Ramanagara",
        "Shimoga": "Shivamogga",
        "Tumkur": "Tumakuru",
        "Udupi *": "Udupi",
    },
    "Kerala": {
        "Kasargod": "Kasaragod",
    },
    "Madhya Pradesh": {
        "Ashok Nagar": "Ashoknagar",
        "Harda *": "Harda",
        "Hoshangabad": "Narmadapuram",
        "Narsimhapur": "Narsinghpur",
        "East Nimar": "Khandwa",        # East Nimar is another name for Khandwa - commented out
        "West Nimar": "Khargone",       # West Nimar is another name for Khargone - commented out
    },
    "Maharashtra": {
        "Ahmadnagar": "Ahmednagar",
        "Ahmed Nagar": "Ahmednagar",
        "Bid": "Beed",
        "Buldana": "Buldhana",
        "Chatrapati Sambhaji Nagar": "Chhatrapati Sambhajinagar",
        "Dharashiv": "Osmanabad",
        "Gondia": "Gondiya",
        "Gondiya *": "Gondiya",
        "Hingoli *": "Hingoli",
        "Mumbai City": "Mumbai",
        "Mumbai( Sub Urban )": "Mumbai Suburban",
        "Nandurbar *": "Nandurbar",
        "Raigarh": "Raigad",
        "Raigarh(Mh)": "Raigad",
        "Washim *": "Washim",
    },

    "Mizoram": {
        "Mammit": "Mamit",
    },
    "Odisha": {
        "Anugal": "Angul",
        "Anugul": "Angul",
        "Baleshwar": "Balasore",
        "Baleswar": "Balasore",
        "Baudh": "Boudh",
        "Jagatsinghapur": "Jagatsinghpur",
        "Jajapur": "Jajpur",
        "Kendrapara *": "Kendrapara",
        "Kendujhar": "Keonjhar",
        "Khorda": "Khordha",
        "Nabarangapur": "Nabarangpur",
        "Sonapur": "Subarnapur",
        "Sundargarh": "Sundargarh",
        "Sundergarh": "Sundargarh",
    },
    "Puducherry": {
        "Pondicherry": "Puducherry",
    },
    "Punjab": {
        "Ferozepur": "Firozpur",
        "S.A.S Nagar(Mohali)": "S.A.S Nagar",
        "Sas Nagar (Mohali)": "S.A.S Nagar",
        "Sri Muktsar Sahib": "Sri Muktsar Sahib",
        "Muktsar":"Sri Muktsar Sahib",
        "Nawanshahr":"Shaheed Bhagat Singh Nagar"
    },
    "Rajasthan": {
        "Chittaurgarh": "Chittorgarh",
        "Dhaulpur": "Dholpur",
        "DeegÂ ": "Deeg",  # remove encoding artifact - cleanup
        "Jhunjhunun": "Jhunjhunu",
        "Jalor": "Jalore",
    },
    "Sikkim": {
        "East": "East Sikkim",
        "North": "North Sikkim",
        "South": "South Sikkim",
        "West": "West Sikkim",

    },
    "Tamil Nadu": {
        "Thiruvallur": "Tiruvallur",
        "Tiruvarur": "Thiruvarur",
        "Tuticorin": "Thoothukkudi",
        "Tirupathur": "Tirupattur",
        "Tiruppur": "Tiruppur",
        "Villupuram": "Viluppuram",
        "Kancheepuram": "Kanchipuram",
        "Kanniyakumari": "Kanyakumari",
        "Namakkal   *": "Namakkal",
},
    "Tripura": {
        "Dhalai  *": "Dhalai",
    },
    "Uttar Pradesh": {
        "Bagpat": "Baghpat",
        "Bara Banki": "Barabanki",
        "Bulandshahar": "Bulandshahr",
        "Faizabad": "Ayodhya",
        "Jyotiba Phule Nagar": "Amroha",
        "Kushi Nagar": "Kushinagar",
        "Kushinagar *": "Kushinagar",
        "Mahrajganj": "Maharajganj",
        "Rae Bareli": "Raebareli",
        "Sant Ravidas Nagar Bhadohi": "Sant Ravidas Nagar",
        "Siddharth Nagar": "Siddharthnagar",
        "Shravasti": "Shrawasti",
    },
    "Uttarakhand": {
        "Hardwar": "Haridwar",
        },
    "West Bengal": {
        "Hooghiy": "Hooghly",
        "Hugli": "Hooghly",
        "Hawrah": "Howrah",
        "Haora": "Howrah",
        "24 Paraganas North": "North 24 Parganas",
        "North Twenty Four Parganas": "North 24 Parganas",
        "24 Paraganas South": "South 24 Parganas",
        "South 24 Pargana": "South 24 Parganas",
        "South Twenty Four Parganas": "South 24 Parganas",
        "North Dinajpur": "Uttar Dinajpur",
        "Dinajpur Dakshin": "Dakshin Dinajpur",
        "Dinajpur Uttar": "Uttar Dinajpur",
        "Coochbehar": "Cooch Behar",
        "Koch Bihar": "Cooch Behar",
        "Medinipur": "Paschim Medinipur",
        "West Medinipur": "Paschim Medinipur",
        "East Midnapore": "Purba Medinipur",
        "East Midnapur": "Purba Medinipur",
        "Medinipur West": "Paschim Medinipur",
        "Barddhaman": "Purba Bardhaman",
        "Bardhaman": "Purba Bardhaman",
        "Burdwan": "Purba Bardhaman",
        "Paschim Bardhaman": "Paschim Bardhaman",
        "West Midnapore": "Paschim Medinipur",
        "South Dinajpur": "Dakshin Dinajpur",
        "Darjiling": "Darjeeling",
        "Maldah": "Malda",
        "Puruliya": "Purulia",
    },
}


In [9]:
# Check if state_map and district_map cover all states and districts from state_district_mapping.csv
import pandas as pd

# Load the state-district mapping
mapping_df = pd.read_csv(r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\state_district_mapping.csv')

# Get unique states and districts
unique_states = mapping_df['state'].unique()
unique_districts_per_state = mapping_df.groupby('state')['district'].unique()

# Check state coverage
all_state_variants = set(state_map.keys()) | set(state_map.values())
uncovered_states = set(unique_states) - all_state_variants
print(f"Uncovered states (not in state_map keys or values): {uncovered_states}")
print(f"Number of uncovered states: {len(uncovered_states)}")

# Check district coverage
uncovered_districts = {}
for state in unique_states:
    if state in district_map:
        canonical_districts = set(district_map[state].keys()) | set(district_map[state].values())
        actual_districts = set(unique_districts_per_state[state])
        uncovered = actual_districts - canonical_districts
        if uncovered:
            uncovered_districts[state] = uncovered

print(f"\nUncovered districts per state (not in district_map[state] keys or values):")
for state, districts in uncovered_districts.items():
    print(f"{state}: {districts}")

if not uncovered_states and not uncovered_districts:
    print("\nAll states and districts from the mapping are covered in the dictionaries.")
else:
    print(f"\nTotal uncovered states: {len(uncovered_states)}, uncovered district groups: {len(uncovered_districts)}")

Uncovered states (not in state_map keys or values): set()
Number of uncovered states: 0

Uncovered districts per state (not in district_map[state] keys or values):
Andhra Pradesh: {'Warangal', 'Krishna', 'Mahbubnagar', 'Dr. B. R. Ambedkar Konaseema', 'Vizianagaram', 'Kurnool', 'Guntur', 'Bapatla', 'Medak', 'Nalgonda', 'Sri Sathya Sai', 'Parvathipuram Manyam', 'Tirupati', 'Karimnagar', 'East Godavari', 'Rangareddi', 'Adilabad', 'Prakasam', 'Srikakulam', 'K.V.Rangareddy', 'Nizamabad', 'K.V. Rangareddy', 'West Godavari', 'Khammam', 'Annamayya', 'Kakinada', 'N. T. R', 'Alluri Sitharama Raju', 'Eluru', 'Palnadu', 'Karim Nagar', 'Chittoor', 'Anakapalli', 'Nandyal', 'Mahabub Nagar', 'Hyderabad', 'Mahabubnagar'}
Assam: {'Majuli', 'Dima Hasao', 'Lakhimpur', 'Biswanath', 'Sribhumi', 'Karbi Anglong', 'Udalguri', 'West Karbi Anglong', 'Darrang', 'Goalpara', 'Bajali', 'Dhemaji', 'South Salmara Mankachar', 'Nalbari', 'Bongaigaon', 'Kamrup Metro', 'Hailakandi', 'Kokrajhar', 'Dibrugarh', 'Jorhat', 'Ti

##Remap districts

In [13]:
# Specific correction for Leh: Move from Jammu and Kashmir to Ladakh and rename to Leh (Ladakh)
import pandas as pd

# Load the final CSV
final_path = r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_merged.csv'
df = pd.read_csv(final_path)

# Correct state for Leh in Jammu and Kashmir
df.loc[(df['state'] == 'Jammu and Kashmir') & (df['district'] == 'Leh'), 'state'] = 'Ladakh'

#  district to Leh (Ladakh) for Ladakh
df.loc[(df['state'] == 'Ladakh') & (df['district'] == 'Leh'), 'district'] = 'Leh (Ladakh)'

# Save back to CSV
df.to_csv(final_path, index=False)

print("Leh correction applied: moved to Ladakh and renamed to Leh (Ladakh).")

Leh correction applied: moved to Ladakh and renamed to Leh (Ladakh).


In [ ]:
# Apply state_map and district_map to the merged CSV
import pandas as pd

# Load the merged CSV
merged_path = r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_merged.csv'
df = pd.read_csv(merged_path)

# Correct erroneous state assignments based on geographical accuracy
df.loc[(df['state'] == 'Andhra Pradesh') & (df['district'].isin(['Khammam', 'Karimnagar', 'Nalgonda', 'Adilabad', 'Warangal', 'Medak', 'Karim Nagar', 'K.V. Rangareddy', 'K.V.Rangareddy', 'Rangareddi', 'Nizamabad', 'Hyderabad', 'Mahabubnagar', 'Mahbub Nagar', 'Mahbubnagar','Mahabub Nagar'])), 'state'] = 'Telangana'
df.loc[(df['state'] == 'Chandigarh') & (df['district'] == 'Rupnagar'), 'state'] = 'Punjab'
df.loc[(df['state'] == 'Jammu and Kashmir') & (df['district'].isin(['Leh (Ladakh)', 'Kargil'])), 'state'] = 'Ladakh'
df.loc[(df['state'] == 'Meghalaya') & (df['district'] == 'Kamrup'), 'state'] = 'Assam'

# Apply state mapping
df['state'] = df['state'].map(state_map).fillna(df['state'])

# Apply district mapping
def map_district(row):
    state = row['state']
    district = row['district']
    if state in district_map and district in district_map[state]:
        return district_map[state][district]
    return district

df['district'] = df.apply(map_district, axis=1)

# Convert date to datetime for proper sorting

# Sort by state, date, district (to have states in alphabetical order first)
df.sort_values(by=['state', 'date', 'district'], inplace=True)

# Convert date back to string for CSV

# Save back to CSV
final_path = r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\merged_final.csv'
df.to_csv(final_path, index=False)

print("Mappings applied, corrections made, and CSV saved as merged_final.csv successfully.")

Mappings applied, corrections made, and CSV saved as merged_final.csv successfully.


##Fixing District Names which appear multiple times across different states by renaming them

In [15]:
import pandas as pd
df = pd.read_csv(r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_merged.csv')
district_states = df.groupby('district')['state'].nunique()
duplicates = district_states[district_states > 1]
print('Districts in multiple states:')
for d in duplicates.index:
     print(f'{d}: {list(df[df["district"] == d]["state"].unique())}')


Districts in multiple states:
Balrampur: ['Chhattisgarh', 'Uttar Pradesh']
Bilaspur: ['Chhattisgarh', 'Himachal Pradesh']
Hamirpur: ['Himachal Pradesh', 'Uttar Pradesh']
Pratapgarh: ['Rajasthan', 'Uttar Pradesh']


In [ ]:
# Disambiguate districts that appear in multiple states
import pandas as pd

# Load the final CSV
final_path = r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\merged_final.csv'
df = pd.read_csv(final_path)

# Define disambiguation mappings
disambiguation = {
    ('Chhattisgarh', 'Balrampur'): 'Balrampur (CG)',
    ('Uttar Pradesh', 'Balrampur'): 'Balrampur (UP)',
    ('Chhattisgarh', 'Bilaspur'): 'Bilaspur (CG)',
    ('Himachal Pradesh', 'Bilaspur'): 'Bilaspur (HP)',
    ('Himachal Pradesh', 'Hamirpur'): 'Hamirpur (HP)',
    ('Uttar Pradesh', 'Hamirpur'): 'Hamirpur (UP)',
    ('Rajasthan', 'Pratapgarh'): 'Pratapgarh (RJ)',
    ('Uttar Pradesh', 'Pratapgarh'): 'Pratapgarh (UP)',
}

# Apply disambiguation
for (state, district), new_district in disambiguation.items():
    df.loc[(df['state'] == state) & (df['district'] == district), 'district'] = new_district

# Save back to CSV
df.to_csv(final_path, index=False)

print("District disambiguation applied and saved to merged_final.csv.")

District disambiguation applied and saved to merged_final.csv.


In [17]:
import pandas as pd
df = pd.read_csv(r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\api_data_aadhar_enrolment_merged.csv')
district_states = df.groupby('district')['state'].nunique()
duplicates = district_states[district_states > 1]
print('Districts in multiple states:')
for d in duplicates.index:
     print(f'{d}: {list(df[df["district"] == d]["state"].unique())}')


Districts in multiple states:


In [ ]:
# Additional district name corrections for Telangana in merged_final.csv
import pandas as pd

# Load the final CSV
final_path = r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\merged_final.csv'
df = pd.read_csv(final_path)
# Additional mappings for Telangana districts
additional_district_map = {
    'K.V. Rangareddy':"Rangareddy",
    "K.V.Rangareddy":"Rangareddy",
    'Rangareddi': 'Rangareddy',  # Map Rangareddi to Rangareddy
    'Mahabub Nagar': 'Mahabubnagar',  # Ensure consistency
    'Mahbubnagar': 'Mahabubnagar',  # Already standard, but included for completeness
    'Karim Nagar':'Karimnagar'
}


# Apply additional district mappings where state is Telangana
df['district'] = df.apply(lambda row: additional_district_map.get(row['district'], row['district']) if row['state'] == 'Telangana' else row['district'], axis=1)

# Save back to CSV
df.to_csv(final_path, index=False)

print("Additional district mappings applied to merged_final.csv successfully.")

Additional district mappings applied to merged_final.csv successfully.


In [ ]:
# Feature Engineering for Demand Forecasting
import pandas as pd
import numpy as np

# Load the preprocessed merged CSV
df = pd.read_csv(r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\merged_final.csv')

# Compute total_enrollment
df['total_enrollment'] = df['age_0_5'] + df['age_5_17'] + df['age_18_greater']

# Ensure date is datetime
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')

# Sort by location and date for time series operations
df = df.sort_values(['state', 'district', 'pincode', 'date']).reset_index(drop=True)

# Time features
df['day_of_week'] = df['date'].dt.dayofweek  # 0=Monday, 6=Sunday
df['week_of_year'] = df['date'].dt.isocalendar().week
df['month'] = df['date'].dt.month
df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
df['is_quarter_end'] = df['date'].dt.is_quarter_end.astype(int)

# Lag features
df['lag_1'] = df.groupby(['state', 'district', 'pincode'])['total_enrollment'].shift(1)
df['lag_7'] = df.groupby(['state', 'district', 'pincode'])['total_enrollment'].shift(7)
df['lag_14'] = df.groupby(['state', 'district', 'pincode'])['total_enrollment'].shift(14)

# Rolling statistics
df['rolling_mean_7'] = df.groupby(['state', 'district', 'pincode'])['total_enrollment'].transform(lambda x: x.rolling(7).mean())
df['rolling_std_7'] = df.groupby(['state', 'district', 'pincode'])['total_enrollment'].transform(lambda x: x.rolling(7).std())

# Growth rate
df['growth_rate_7'] = (df['total_enrollment'] - df['lag_7']) / df['lag_7']
df['growth_rate_7'] = df['growth_rate_7'].replace([np.inf, -np.inf], np.nan)  # Handle division by zero

# Categorical encoding
# Target encoding for state and district (mean total_enrollment)
state_mean = df.groupby('state')['total_enrollment'].mean()
df['state_encoded'] = df['state'].map(state_mean)

district_mean = df.groupby('district')['total_enrollment'].mean()
df['district_encoded'] = df['district'].map(district_mean)

# Frequency encoding for pincode
pincode_freq = df['pincode'].value_counts()
df['pincode_encoded'] = df['pincode'].map(pincode_freq)

# Compute target: total_enrollment_next_7_days (sum of next 7 days' total_enrollment)
df['total_enrollment_next_7_days'] = (
    df.groupby(['state', 'district', 'pincode'])['total_enrollment'].shift(-1) +
    df.groupby(['state', 'district', 'pincode'])['total_enrollment'].shift(-2) +
    df.groupby(['state', 'district', 'pincode'])['total_enrollment'].shift(-3) +
    df.groupby(['state', 'district', 'pincode'])['total_enrollment'].shift(-4) +
    df.groupby(['state', 'district', 'pincode'])['total_enrollment'].shift(-5) +
    df.groupby(['state', 'district', 'pincode'])['total_enrollment'].shift(-6) +
    df.groupby(['state', 'district', 'pincode'])['total_enrollment'].shift(-7)
)

# Drop rows with NaN in target or essential features (due to lags)
df = df.dropna(subset=['total_enrollment_next_7_days', 'lag_1', 'lag_7', 'lag_14'])

# Save the engineered dataset
df.to_csv(r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\merged_final.csv', index=False)

print("Feature engineering completed. Shape:", df.shape)
print("Columns:", df.columns.tolist())
print(df.head())

Feature engineering completed. Shape: (539593, 23)
Columns: ['date', 'state', 'district', 'pincode', 'age_0_5', 'age_5_17', 'age_18_greater', 'total_enrollment', 'day_of_week', 'week_of_year', 'month', 'is_month_end', 'is_quarter_end', 'lag_1', 'lag_7', 'lag_14', 'rolling_mean_7', 'rolling_std_7', 'growth_rate_7', 'state_encoded', 'district_encoded', 'pincode_encoded', 'total_enrollment_next_7_days']
         date                        state district  pincode  age_0_5  \
79 2025-09-23  Andaman And Nicobar Islands  Nicobar   744301        1   
80 2025-09-24  Andaman And Nicobar Islands  Nicobar   744301        0   
81 2025-09-25  Andaman And Nicobar Islands  Nicobar   744301        1   
82 2025-10-17  Andaman And Nicobar Islands  Nicobar   744301        3   
83 2025-10-24  Andaman And Nicobar Islands  Nicobar   744301        1   

    age_5_17  age_18_greater  total_enrollment  day_of_week  week_of_year  \
79         0               0                 1            1            39   
80 

In [20]:
df.shape

(1006007, 7)

##*Dropping* Duplicates

In [ ]:

# Check for Exact Duplicates (Whole Row Match)
import pandas as pd

# Load the merged dataset
df = pd.read_csv(r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\merged_final.csv')
print('Total rows before dropping duplicates:', len(df))
df.drop_duplicates(inplace=True)
df.to_csv(r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\merged_final.csv', index=False)

print('Total rows after dropping duplicates:', len(df))

# Check for exact duplicates (all columns identical)
exact_duplicates = df.duplicated(keep=False)
print('Number of exact duplicate rows (including originals):', exact_duplicates.sum())

if exact_duplicates.sum() > 0:
    print('\nExact duplicate groups (count per group):')
    # Group by all columns to count exact duplicates
    exact_dup_groups = df[exact_duplicates].groupby(list(df.columns)).size()
    print(f"Number of unique exact duplicate groups: {len(exact_dup_groups)}")
    print(f"Total exact duplicates: {exact_dup_groups.sum()}")

    print('\nSample of exact duplicate pairs (showing first 3 examples):')
    for i, ((group_data), count) in enumerate(exact_dup_groups.head(3).items()):
        print(f"\nExact duplicate group {i+1} (appears {count} times):")
        # Find the rows that match this group
        mask = (df == group_data).all(axis=1)
        print(df[mask].head(count))
else:
    print('No exact duplicates found (no rows are completely identical).')

Total rows before dropping duplicates: 1006007
Total rows after dropping duplicates: 973808
Number of exact duplicate rows (including originals): 0
No exact duplicates found (no rows are completely identical).


In [ ]:
# Find Districts with Same Pincode
import pandas as pd

# Load the merged dataset
df = pd.read_csv(r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\merged_final.csv')

print('Total rows:', len(df))

# Group by pincode and find unique districts for each pincode
pincode_districts = df.groupby('pincode')['district'].unique()

# Filter for pincodes that have multiple districts
shared_pincodes = pincode_districts[pincode_districts.apply(len) > 1]

print(f"\nNumber of pincodes shared across multiple districts: {len(shared_pincodes)}")

if len(shared_pincodes) > 0:
    print("\nShared pincodes and their districts:")
    for pincode, districts in shared_pincodes.items():
        print(f"Pincode {pincode}: {list(districts)}")

    print(f"\nTotal unique districts involved: {len(set(district for districts in shared_pincodes for district in districts))}")

    # Show some statistics
    district_counts = shared_pincodes.apply(len)
    print(f"\nDistribution of shared pincodes:")
    print(f"Most shared: {district_counts.max()} districts per pincode")
    print(f"Average: {district_counts.mean():.1f} districts per pincode")

    # Show sample of the most shared pincodes
    most_shared = shared_pincodes[district_counts == district_counts.max()]
    if len(most_shared) > 0:
        print(f"\nExample of most shared pincode:")
        pincode = most_shared.index[0]
        districts = most_shared.iloc[0]
        print(f"Pincode {pincode} is used by {len(districts)} districts: {list(districts)}")
else:
    print("No pincodes are shared across multiple districts.")

Total rows: 973808

Number of pincodes shared across multiple districts: 4125

Shared pincodes and their districts:
Pincode 110001: ['Central Delhi', 'New Delhi']
Pincode 110003: ['Central Delhi', 'New Delhi', 'South Delhi']
Pincode 110007: ['Central Delhi', 'North Delhi']
Pincode 110011: ['Central Delhi', 'New Delhi']
Pincode 110016: ['South Delhi', 'South West Delhi']
Pincode 110021: ['New Delhi', 'South West Delhi']
Pincode 110023: ['New Delhi', 'South West Delhi']
Pincode 110025: ['South Delhi', 'South East Delhi']
Pincode 110029: ['South Delhi', 'South West Delhi']
Pincode 110030: ['South Delhi', 'South West Delhi']
Pincode 110032: ['Shahdara', 'East Delhi', 'North East Delhi']
Pincode 110037: ['South West Delhi', 'New Delhi']
Pincode 110039: ['North Delhi', 'North West Delhi']
Pincode 110040: ['North West Delhi', 'North Delhi']
Pincode 110043: ['South West Delhi', 'West Delhi', 'Najafgarh']
Pincode 110047: ['South Delhi', 'South West Delhi']
Pincode 110053: ['North East Delhi', '

In [ ]:
import pandas as pd

# Load the final CSV
final_path = r'S:\UIDAI_Hackathon\api_data_aadhar_enrolment\merged_final.csv'
df = pd.read_csv(final_path)

# Group by state and count unique districts
state_district_counts = df.groupby('state')['district'].nunique().sort_index()

print("States and their district counts:")
for state, count in state_district_counts.items():
    print(f"{state}: {count} districts")

print(f"\nTotal states: {len(state_district_counts)}")
print(f"Total districts overall: {state_district_counts.sum()}")

States and their district counts:
Andaman and Nicobar Islands: 4 districts
Andhra Pradesh: 26 districts
Arunachal Pradesh: 25 districts
Assam: 37 districts
Bihar: 39 districts
Chandigarh: 1 districts
Chhattisgarh: 36 districts
Dadra and Nagar Haveli and Daman and Diu: 3 districts
Delhi: 12 districts
Goa: 3 districts
Gujarat: 34 districts
Haryana: 22 districts
Himachal Pradesh: 13 districts
Jammu and Kashmir: 20 districts
Jharkhand: 24 districts
Karnataka: 34 districts
Kerala: 14 districts
Ladakh: 2 districts
Lakshadweep: 1 districts
Madhya Pradesh: 55 districts
Maharashtra: 38 districts
Manipur: 12 districts
Meghalaya: 13 districts
Mizoram: 11 districts
Nagaland: 17 districts
Odisha: 30 districts
Puducherry: 3 districts
Punjab: 23 districts
Rajasthan: 39 districts
Sikkim: 6 districts
Tamil Nadu: 38 districts
Telangana: 35 districts
Tripura: 8 districts
Uttar Pradesh: 77 districts
Uttarakhand: 14 districts
West Bengal: 23 districts

Total states: 36
Total districts overall: 792


In [54]:
df.shape

(973808, 7)